#Hard coded values
Column names
'createdatl,'updatedatl','marketproduct_id','Eddress ID','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','list_price,'g10_price','transfer_groups','isvisible'

Segment filter
"Asset", "Delist", "None", "Test", "Only Promo"

category managers
"Marius","Fabian","Gina","Hamid","Amadeus","Rozan","Francisco","Eike","Evrim","Leonard","Andrea"

Cities
"Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen","Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München", "Nürnberg/Fürth","Frankfurt","Offenbach","Stuttgart","Heidelberg","Mannheim","Karlsruhe","Darmstadt","Augsburg"

segments	segment_name
1	Test
10	Seasonal
11	Supply Shortage
12	Stand By
2	Launch
3	Regular
4	Regional
5	Only Promo
6	Delist On Depletion
7	Delist
8	Asset


final output columns format


#Using Instructions
1 - save panel report on desktop and rename until date. (ex - G10andGetirBuyuk_AllProducts_without_TRDE_20230605.xlsx)
2 - save eddress export all on desktop and rename as Inventory.xls
3 - run the code - you will have a sheet on your desktop "New_assortment"
4 - copy the data from "New_assortment" and paste it in Assortment list sheet - "Weekly updated Assortment List"




In [4]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
today = dt.now()

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)


#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#formatting based on the required data type
Assortment['createdatl'] = Assortment['createdatl'].dt.date
Assortment['inbox_quantitiy'] = Assortment['inbox_quantitiy'].fillna(0)
Assortment['inbox_quantitiy'] = Assortment['inbox_quantitiy'].astype('int')

#merge data from multiple sheets from product data report
Ing = pd.read_excel(Path_A, "DE Product Descriptions")
Ing = Ing[['_id','ingredients_en','displayName_en','shortDescription_en']]
Ing.rename(columns={'_id':'Getir Product Id'}, inplace=True)

Assortment_Joined = pd.merge(Assortment, 
                      Ing, 
                      on ='Getir Product Id', 
                      how ='left')

#Assortment_Joined = Assortment.copy()

#------------------------------


Category_dictionary = {\
"Rozan":['Coffee & Tea'],
"Marius":['Baby','Beer','Home Care','Personal Care','Pets','Spirits & Co.','Wine & Co.'],
"Leonard":['Fruit & Veg'],
"Hamid":['Cheese & Co.','Milk & Eggs','Yoghurt & Co.','Cold Cuts','Meat & Fish'],
"Francisco":['Fresh & Ready'],
"Fabian":['Bakery'],
"Evrim":['Frozen Food','Ice Cream'],
"Eike":['Drinks','Breakfast','Pantry','Pasta & Rice','Canned','Crisps & Snacks'],
"Amadeus":['Christmas','Easter','Sweet Snacks'],
"Other":['0','DUMMY TEST KATEGORI GE',"Mother's Day",'Top Deals','Earthquake Aid','Dairy','Cheese & Deli','Vegan & Veggie','Confectionery','Vegan']}


# #------------------------------

 #match the categories and get the name of category manager
for ind, row in Assortment_Joined.iterrows():
    if Assortment_Joined.loc[ind, 'category_name_en'] == "":
        Assortment_Joined.loc[ind, 'Buyer'] = "Not Assigned"
    else:
        Category = Assortment_Joined.loc[ind, 'category_name_en']
        Assortment_Joined.loc[ind, 'Buyer'] = [k for k, v in Category_dictionary.items() if Category in v]


#get only the name without brackets
Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].astype(str)
Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].str[2:-2]

#------------------------------

#Generate the y/n combination for transfer group(mfc) checks
#Assortment_Joined['transfer_groups'] = Assortment_Joined['transfer_groups'].astype(str)

#create a list of cities 
cities = ["Berlin","Berlin mini", "Dresden","Leipzig","Hamburg","Bremen",\
          "Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München",\
          "NürnbergFürth","Frankfurt","Offenbach","Stuttgart","Heidelberg",\
          "Mannheim","Karlsruhe","Darmstadt","Augsburg"]

Assortment_Joined['Cities'] = ''
#iterate over transfer groups to check for cities
for c in cities:
    for ind, row in Assortment_Joined.iterrows():
        x = "merged "+c
        if c in str(Assortment_Joined.loc[ind, 'transfer_groups']):
            a = Assortment_Joined.loc[ind, 'Cities']
            b = c
            new_Cit = a+", "+b
            Assortment_Joined.loc[ind, 'Cities'] = new_Cit
        else:
            pass
            
Assortment_Joined['Cities'] = Assortment_Joined['Cities'].str[2:]

#-------------------------------
#Extract the nutrients from the downloaded format and merge them with Assortment

Path_B = "/Users/swaroopbasavarajmusti/Desktop/Export_Data.xlsx"
Nut_All = pd.read_excel(Path_B)
Nut_Extract = Nut_All.copy()

#join the name UOM with nutrients value and pivot
Nut_Extract['itemValue'] = Nut_Extract['itemValue.en'].astype(str) +" "+ Nut_Extract['itemUnit.en'].astype(str)


Nut_Extract = Nut_Extract.pivot_table(index=['_id','sectionTitle.en'],columns='itemName.en',values = 'itemValue',aggfunc='first').\
                reset_index()


#extract the UOM from values and create new columns
Nut_Extract['Calories.UOM'] = Nut_Extract['Calories'].str.split(' ').str[1]
Nut_Extract['Energy.UOM'] = Nut_Extract['Energy'].str.split(' ').str[1]
Nut_Extract['Fat.UOM'] = Nut_Extract['Fat'].str.split(' ').str[1]
Nut_Extract['of which saturated fat.UOM'] = Nut_Extract['of which saturated fat'].str.split(' ').str[1]
Nut_Extract['Carbohydrate.UOM'] = Nut_Extract['Carbohydrate'].str.split(' ').str[1]
Nut_Extract['of which sugars.UOM'] = Nut_Extract['of which sugars'].str.split(' ').str[1]
Nut_Extract['Protein.UOM'] = Nut_Extract['Protein'].str.split(' ').str[1]
Nut_Extract['Salt.UOM'] = Nut_Extract['Salt'].str.split(' ').str[1]

Nut_Extract['Calories'] = Nut_Extract['Calories'].str.split(' ').str[0]
Nut_Extract['Energy'] = Nut_Extract['Energy'].str.split(' ').str[0]
Nut_Extract['Fat'] = Nut_Extract['Fat'].str.split(' ').str[0]
Nut_Extract['of which saturated fat'] = Nut_Extract['of which saturated fat'].str.split(' ').str[0]
Nut_Extract['Carbohydrate'] = Nut_Extract['Carbohydrate'].str.split(' ').str[0]
Nut_Extract['of which sugars'] = Nut_Extract['of which sugars'].str.split(' ').str[0]
Nut_Extract['Protein'] = Nut_Extract['Protein'].str.split(' ').str[0]
Nut_Extract['Salt'] = Nut_Extract['Salt'].str.split(' ').str[0]


#select only required columns
Nut_Extract =Nut_Extract[['_id','sectionTitle.en',"Calories",'Calories.UOM','Energy','Energy.UOM','Fat','Fat.UOM',\
                          'of which saturated fat','of which saturated fat.UOM','Carbohydrate','Carbohydrate.UOM',\
                          'of which sugars','of which sugars.UOM','Protein','Protein.UOM','Salt','Salt.UOM']]
Nut_Extract.rename(columns={'_id':'Getir Product Id'}, inplace=True)

#Merge the nutrients dataframe to assortment on ids
Assortment_with_nutrients = pd.merge(Assortment_Joined, 
                      Nut_Extract, 
                      on ='Getir Product Id', 
                      how ='left')

#------------------------------

#creating dictionary to match the product ids

Path_C = "/Users/swaroopbasavarajmusti/Desktop/Inventory.xls"
Dictionary = pd.read_excel(Path_C)

Dictionary = Dictionary[Dictionary['Getir Product Id'].notna()].reset_index(drop = True)
Dictionary = Dictionary[['Getir Product Id','Product Id','Product SKU']]

#------------------------------

#left joining Panel assortment with eddress ID dictionary

Assortment_with_nutrients_V3 = pd.merge(Assortment_with_nutrients,Dictionary,on ='Getir Product Id',how ='left')

Assortment_V3 = Assortment_with_nutrients_V3.copy()

Assortment_V3['Allergens'] = ''
#-------------------------------


# Allergens derivation

Path_G = "/Users/swaroopbasavarajmusti/Desktop/Allergens.xlsx"

Allergens_raw = pd.read_excel(Path_G)

Allergens = Allergens_raw.copy()

#look up for ingredients and create a attribute for allergens

for ind,row in Assortment_V3.iterrows():
    for S,R in Allergens.iterrows():
        if str(Allergens.loc[S,'Search_Value']) in str(Assortment_V3.loc[ind,'ingredients_en']):
            if Assortment_V3.loc[ind,'Allergens'] == "":
                Assortment_V3.loc[ind,'Allergens'] = Allergens.loc[S,'Return_Value']
            else:
                x = Assortment_V3.loc[ind,'Allergens']
                y = Allergens.loc[S,'Return_Value']
                z = x+","+y
                Assortment_V3.loc[ind,'Allergens'] = z
        elif str(Allergens.loc[S,'Search_Value']).lower() in str(Assortment_V3.loc[ind,'ingredients_en']):
            if Assortment_V3.loc[ind,'Allergens'] == "":
                Assortment_V3.loc[ind,'Allergens'] = Allergens.loc[S,'Return_Value']
            else:
                x = Assortment_V3.loc[ind,'Allergens']
                y = Allergens.loc[S,'Return_Value']
                z = x+","+y
                Assortment_V3.loc[ind,'Allergens'] = z
        else:
            pass
        

#Create a ouptut to excel
#Assortment_V3.to_excel("/Users/swaroopbasavarajmusti/Desktop/Assortment_V3.xlsx")

#--------------------------------------------

#Automatically save the data into master assortment with API connections

import warnings
warnings.simplefilter("ignore")
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file', \
        'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('panel-321812-ac5afc8690ea.json',scope)
client = gspread.authorize(creds)

Data_repo  = client.open('V3_Master_Assortment_with_nutrients').worksheet('Master_data')
Data_repo.clear()

Assortment_V3 = Assortment_V3[['Getir Product Id',
'parentproduct_name_en',
'displayName_en',
'shortDescription_en',
'agerestriction',
'brand_name',
'manufacturer_name',
'supplier_names',
'category_name_en',
'mastercategoriesv2_name_en',
'list_price',
'g10_price',
'g10_struck_price',
'vat',
'picurl_en',
'exp_days_allowed',
'exp_days_dead',
'createdatl',
'transfer_groups',
'product_barcode',
'product_dimension_depth',
'product_dimension_height',
'product_dimension_width',
'product_net_weight',
'product_gross_weight',
'inbox_barcode',
'inbox_quantitiy',
'inbox_dimension_depth',
'inbox_dimension_height',
'inbox_dimension_width',
'inbox_net_weight',
'inbox_gross_weight',
'mainstorespecs_shipmenttype',
'palet_inbox_quantity',
'palet_height',
'palet_gross_weight',
'deposit_price',
'Buyer',
'Cities',
'sectionTitle.en',
'Calories',
'Calories.UOM',
'Energy',
'Energy.UOM',
'Fat',
'Fat.UOM',
'of which saturated fat',
'of which saturated fat.UOM',
'Carbohydrate',
'Carbohydrate.UOM',
'of which sugars',
'of which sugars.UOM',
'Protein',
'Protein.UOM',
'Salt',
'Salt.UOM',
'Allergens']]

Assortment_V3 = Assortment_V3.fillna('').astype(str)
#Assortment_V3 = Assortment_V3.fillna('')
Data_repo.update([Assortment_V3.columns.values.tolist()] + Assortment_V3.values.tolist())

{'spreadsheetId': '1SIBJ7abHJ2valBAdexvk6hEJUzVvz42nxK5gzlCImPU',
 'updatedRange': 'Master_data!A1:BE7286',
 'updatedRows': 7286,
 'updatedColumns': 57,
 'updatedCells': 415302}